In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import losses, backend
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
# # Check for classes distribution
train_target = pd.read_csv("../input/lish-moa/train_targets_scored.csv")

In [ ]:

test = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")


In [ ]:
ctlVehicle_idx = train["cp_type"] != "ctl_vehicle"
train = train.loc[ctlVehicle_idx].reset_index(drop=True)
train = train.drop("cp_type", axis=1)
train_target = train_target.loc[ctlVehicle_idx].reset_index(drop=True)

In [ ]:
gcols = [g for g in train.columns if "g-" in g]
ccols = [c for c in train.columns if "c-" in c]
cpcols = [cp for cp in train.columns if "cp_" in cp]

In [ ]:

ctlVehicle_test = test["cp_type"] == "ctl_vehicle"
test = test.drop("cp_type", axis=1)

# enc = LabelEncoder()
# for col in train[cpcols]:
#     train[col] = enc.fit_transform(train[col])
    
# enc = LabelEncoder()
# for col in test[cpcols]:
#     test[col] = enc.fit_transform(test[col])
train["cp_time"]=train["cp_time"].map({24: 1, 48: 2, 72: 3})
train["cp_dose"] = train["cp_dose"].map({"D1": 1, "D2": 2})
test["cp_time"]= test["cp_time"].map({24: 1, 48: 2, 72: 3})
test["cp_dose"]=test["cp_dose"].map({"D1": 1, "D2": 2})
train

In [ ]:
for col in train.iloc[:, 3:].columns:
    percent = train[col].quantile([0.01, 0.99]).values
    train[col] = np.clip(train[col], percent[0], percent[1])

for col in test.iloc[:, 3:].columns:
    percent = test[col].quantile([0.01, 0.99]).values
    test[col] = np.clip(test[col], percent[0], percent[1])

In [ ]:
from sklearn.decomposition import PCA

g_pca = PCA(n_components=0.99)
c_pca = PCA(n_components=0.99)

train_test_g_concat = pd.concat([train[gcols], test[gcols]], axis=0)
train_test_c_concat = pd.concat([train[ccols], test[ccols]], axis=0)
g_pca.fit(train_test_g_concat)
c_pca.fit(train_test_c_concat)

train_gtrans = pd.DataFrame(g_pca.transform(train[gcols]), columns=["g_PCA" + str(i) for i in range(g_pca.n_components_)], index=train.index)
test_gtrans = pd.DataFrame(g_pca.transform(test[gcols]), columns=["g_PCA" + str(i) for i in range(g_pca.n_components_)], index=test.index)

train_ctrans = pd.DataFrame(c_pca.transform(train[ccols]), columns=["c_PCA" + str(i) for i in range(c_pca.n_components_)], index=train.index)
test_ctrans = pd.DataFrame(c_pca.transform(test[ccols]), columns=["c_PCA" + str(i) for i in range(c_pca.n_components_)], index=test.index)

g_pca.n_components_, c_pca.n_components_

In [ ]:
train = pd.concat([train_gtrans, train_ctrans, train[cpcols]], axis=1)
test = pd.concat([test_gtrans, test_ctrans, test[cpcols]], axis=1)
train

In [ ]:


train_target = train_target.drop("sig_id", axis=1)
train_target

In [ ]:
def create_shallow_model():
    model = tf.keras.Sequential([
        tfa.layers.WeightNormalization(L.Dense(train.shape[1], input_shape=(train.shape[1],))),
        L.BatchNormalization(),
        L.Dropout(0.2),
        tfa.layers.WeightNormalization(L.Dense(128, activation="relu")),
        L.BatchNormalization(),
        L.Dropout(0.2),
        tfa.layers.WeightNormalization(L.Dense(train_target.shape[1], activation="sigmoid"))
    ])
    
    sgd = tf.keras.optimizers.SGD()
    adamw = tfa.optimizers.AdamW(weight_decay = 0.0001)
    adam = tf.keras.optimizers.Adam()
    radam = tfa.optimizers.RectifiedAdam()
    lookahead_radam = tfa.optimizers.Lookahead(radam)
    lookahead_adamw = tfa.optimizers.Lookahead(adamw)
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-6), optimizer=adam)
    return model

In [ ]:
def create_mid_model():
    model = tf.keras.Sequential([
        tfa.layers.WeightNormalization(L.Dense(train.shape[1], input_shape=(train.shape[1],))),
        L.BatchNormalization(),
        tfa.layers.WeightNormalization(L.Dense(512, activation="relu")),
        L.BatchNormalization(),
        L.Dropout(0.3),
        tfa.layers.WeightNormalization(L.Dense(512, activation="relu")),
        L.BatchNormalization(),
        L.Dropout(0.3),
        tfa.layers.WeightNormalization(L.Dense(train_target.shape[1], activation="sigmoid"))
    ])
    
    sgd = tf.keras.optimizers.SGD()
    adamw = tfa.optimizers.AdamW(weight_decay = 0.0001)
    adam = tf.keras.optimizers.Adam()
    radam = tfa.optimizers.RectifiedAdam()
    lookahead_radam = tfa.optimizers.Lookahead(radam)
    lookahead_adamw = tfa.optimizers.Lookahead(adamw)
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-6), optimizer=lookahead_adamw)
    return model

In [ ]:
def create_deep_model():
    model = tf.keras.Sequential([
        tfa.layers.WeightNormalization(L.Dense(train.shape[1], input_shape=(train.shape[1],))),
        L.BatchNormalization(),
        tfa.layers.WeightNormalization(L.Dense(512, activation="selu")),
        L.BatchNormalization(),
        L.Dropout(0.42),
        tfa.layers.WeightNormalization(L.Dense(512, activation="relu")),
        L.BatchNormalization(),
        L.Dropout(0.42),
        tfa.layers.WeightNormalization(L.Dense(256, activation="relu")),
        L.BatchNormalization(),
        L.Dropout(0.42),
        tfa.layers.WeightNormalization(L.Dense(256, activation="elu")),
        L.BatchNormalization(),
        L.Dropout(0.1),
        tfa.layers.WeightNormalization(L.Dense(train_target.shape[1], activation="sigmoid"))
    ])
    
    sgd = tf.keras.optimizers.SGD()
    adamw = tfa.optimizers.AdamW(weight_decay = 0.0001)
    adam = tf.keras.optimizers.Adam()
    radam = tfa.optimizers.RectifiedAdam()
    lookahead_radam = tfa.optimizers.Lookahead(radam)
    lookahead_adamw = tfa.optimizers.Lookahead(adamw)
    
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-6), optimizer=lookahead_radam)
    return model

In [ ]:

predictions = []
kf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
tf.random.set_seed(42)
for fold_id, (train_idx, valid_idx) in enumerate(kf.split(train, train_target)):
    model1 = create_shallow_model()
    model2 = create_mid_model()
    model3 = create_deep_model()
    
    history1 = model1.fit(train.iloc[train_idx], train_target.iloc[train_idx], batch_size=32,
              validation_data=(train.iloc[valid_idx], train_target.iloc[valid_idx]),
             epochs=100,
             verbose=2,
             callbacks=[
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("model1_fold" + str(fold_id) + ".h5", save_best_only=True, save_weights_only=True)
])
    print("Model 1, Fold ID: {}, train loss: {}, valid loss: {}".format(fold_id, min(history1.history["loss"]), min(history1.history["val_loss"])))
    history2 = model2.fit(train.iloc[train_idx], train_target.iloc[train_idx], batch_size=32,
              validation_data=(train.iloc[valid_idx], train_target.iloc[valid_idx]),
             epochs=100,
             verbose=2,
             callbacks=[
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("model2_fold" + str(fold_id) + ".h5", save_best_only=True, save_weights_only=True)
])
    print("Model 2, Fold ID: {}, train loss: {}, valid loss: {}".format(fold_id, min(history2.history["loss"]), min(history2.history["val_loss"])))
    history3 = model3.fit(train.iloc[train_idx], train_target.iloc[train_idx], batch_size=32,
              validation_data=(train.iloc[valid_idx], train_target.iloc[valid_idx]),
             epochs=100,
             verbose=2,
             callbacks=[
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("model3_fold" + str(fold_id) + ".h5", save_best_only=True, save_weights_only=True)
])
    
    print("Model 3, Fold ID: {}, train loss: {}, valid loss: {}".format(fold_id, min(history3.history["loss"]), min(history3.history["val_loss"])))
    
    model1.load_weights("model1_fold" + str(fold_id) + ".h5")
    model2.load_weights("model2_fold" + str(fold_id) + ".h5")
    model3.load_weights("model3_fold" + str(fold_id) + ".h5")
    pred1 = model1.predict(test)
    pred2 = model2.predict(test)
    pred3 = model3.predict(test)
    predictions.append(np.average([pred1, pred2, pred3], weights=[0.15, 0.7, 0.15], axis=0))

In [ ]:
pred = np.mean(predictions, axis=0)
pred = np.clip(pred, 0.001, 0.999)
pred.shape

In [ ]:
sub = pd.read_csv("../input/lish-moa/sample_submission.csv")
sub.loc[:, 1:] = pred
sub.loc[ctlVehicle_test, sub.columns != "sig_id"] = 0


In [ ]:
sub.to_csv("submission.csv", index=False)